# Convolutional Neural Networks - CIFAR10

## Load Data

In [ ]:
import numpy as np
import tensorflow as tf
%run -i tools.py

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Convert y_train, y_test to be a list
train_labels = [y for arr in train_labels for y in arr]
test_labels = [y for arr in test_labels for y in arr]

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [ ]:
print("Train images:", train_images.shape)
print("Train labels:", len(train_labels)) 
print("Test images:", test_images.shape)
print("Test labels", len(test_labels))
print("Number of classes:", len(classes))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def show_image(index):
    print(classes[train_labels[index]])
    plot = plt.imshow(train_images[index])

In [ ]:
idx = 1000 
show_image(idx)

## Data Preprocessing

In [ ]:
import keras
from keras.utils import to_categorical
from sklearn.utils import shuffle

X_train = train_images.astype('float32')
X_test = test_images.astype('float32')

# Normalize Features: Convert Pixels from the range 0-255 to 0-1
X_train /= 255.
X_test /= 255.

# One-Hot Encoding for Labels: ['cat', 'dog'] => [(0,1), (1,0)]
y_train = keras.utils.to_categorical(train_labels)
y_test = keras.utils.to_categorical(test_labels)

# Shuffle Data
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [ ]:
# Shorten Dataset (We would use all data in practice, but we don't have enough time :)
PERCENT_DATA_USED = .10 

num_train = int(PERCENT_DATA_USED * len(X_train))
num_test = int(PERCENT_DATA_USED * len(X_test))
X_train = X_train[:num_train]
y_train = y_train[:num_train]
X_test = X_test[:num_test]
y_test = y_test[:num_test]

print(len(X_train), len(y_train), len(X_test), len(y_test))

## Create CNN Model

- [Convolutional Layers](https://keras.io/layers/convolutional/) 
    - filters: number of kernals (feature_detectors) in the layer
    - kernal_size: size of the kernal
    - activation: relu activation function typically used
    - padding: how to perserve information in borders
    - strides: how to slide kernal over the input (step size)
- [Pooling layers](https://keras.io/layers/pooling/)
   - max pooling: takes only the maximum pixel value from a region of pixels
   - pool_size: region of pixels to consider
- [Dropout](https://keras.io/layers/core/#dropout) 
    - randomly drop neurons during training, helps prevent overfitting
    - rate: percentage of neurons to drop
- [Batch norm layer](https://keras.io/layers/normalization/)
    - normalizes data at a given point in the network, helps prevent overfitting
- [Dense Layer](https://keras.io/layers/core/#dense) 
    - regular fully-connected layer in traditional NN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5, 5), kernel_initializer='he_uniform', activation='relu', input_shape=(32, 32, 3), padding='same'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='relu', padding='same'))  
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))